### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torchvision
# 禁止显示Beta版transforms的警告
torchvision.disable_beta_transforms_warning()
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.transforms.v2 as v2
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision.datasets import ImageFolder
import shutil
import torch.optim as optim
from PIL import Image
from tqdm import tqdm
from glob import glob
from collections import Counter
import copy
import matplotlib.pyplot as plt

### Set Device

In [2]:
# CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
print(torch.cuda.is_available())

cuda
True


### Data Preprocessing

In [3]:
transform = v2.Compose([
                v2.Resize((256, 256)),
                v2.RandomCrop((150, 150)),
                
                v2.RandomHorizontalFlip(p=0.5),
                v2.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=(0, 10)),
                v2.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    
                v2.ToTensor(), # If you have problem with ToTensor, try ToImageTensor
                # v2.ToImageTensor(),
                v2.ConvertImageDtype(),
                v2.Normalize((0.425, 0.415, 0.405), (0.205, 0.205, 0.205))
                # v2.Normalize((0.4307, 0.4577, 0.4541), (0.2363, 0.2348, 0.2434))
])

transform_test = v2.Compose([
                v2.Resize((150,150)),
                
                v2.ToTensor(), # If you have problem with ToTensor, try ToImageTensor
                # v2.ToImageTensor(),
                v2.ConvertImageDtype(),
                v2.Normalize((0.425, 0.415, 0.405), (0.205, 0.205, 0.205))
                # v2.Normalize((0.4307, 0.4577, 0.4541), (0.2363, 0.2348, 0.2434))
])

d:\Anaconda\envs\FLlearner\lib\site-packages\torchvision\transforms\v2\_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


### Set Data Path

In [4]:
# Change to your folder pathes here
root_path = './'

train_dir = root_path + 'data_demo/train'
validation_dir = root_path + 'data_demo/validation'
test_dir = root_path + 'data_demo/test'

# Set the nums of each class you wish the validation set has!
nums_per_class = 6
classes = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]

print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


### Define Dataset

In [5]:
# Load and pack the training set and validation set
class MyDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
            img_path, class_category = self.dataset[idx]
            image = Image.open(img_path)
            if self.transform:
                image = self.transform(image)
            return idx, image, class_category, img_path


train = ImageFolder(train_dir)
validation = ImageFolder(validation_dir)

train_data, train_label = train.imgs, train.targets
validation_data, validation_label = validation.imgs, validation.targets

train_dataset = MyDataset(train_data, transform)
validation_dataset = MyDataset(validation_data, transform_test)

In [6]:
# Label mapping
label_mapping = train.class_to_idx
print(label_mapping)

{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


In [7]:
# Load and pack the test set
class TestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_paths = [os.path.join(folder_path, img_name) for img_name in os.listdir(folder_path) if img_name.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, img_path

test_dataset = TestDataset(test_dir, transform_test)

In [8]:
class DscDataset(Dataset):
    def __init__(self, features, labels):
        # 转换成Tensor，为了方便后续处理
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [9]:
# Interface of Active Learning Dataset
class ActiveLearningDataset:
    def __init__(self, dataset):
        self.dataset = dataset
        self.n_pool = len(dataset)
        self.labeled_idxs = np.zeros(self.n_pool, dtype=bool) # labeled_idxs = True means this data has been labeled

    def initialize_labels(self, num):
        np.random.seed(42) # 42 is the fixed number here, don't change it!
        tmp_idxs = np.arange(self.n_pool)
        np.random.shuffle(tmp_idxs)
        self.labeled_idxs[tmp_idxs[:num]] = True
        
    def set_labeled_true(self, path):
        idx = 0
        for img, _, img_path in self.dataset:
            if img_path is path:
                self.labeled_idxs[idx] = True
                print("TMD Ok")
                break
            idx+=1

    def get_labeled_data(self):
        labeled_idxs = np.where(self.labeled_idxs)[0]
        return Subset(self.dataset, labeled_idxs), labeled_idxs

    def get_unlabeled_data(self):
        unlabeled_idxs = np.where(~self.labeled_idxs)[0]
        return Subset(self.dataset, unlabeled_idxs), unlabeled_idxs
    
    def get_required_unlabeled_data(self, num):
        unlabeled_idxs = np.where(~self.labeled_idxs)[0]
        if len(unlabeled_idxs) < num:
            raise ValueError(f"Requested {num} unlabeled data points, but only {len(unlabeled_idxs)} are available.")
        selected_idxs = np.random.choice(unlabeled_idxs, num, replace=False)
        return Subset(self.dataset, selected_idxs), selected_idxs

    def copy_with_updates(self):
        new_dataset = ActiveLearningDataset(self.dataset)
        new_dataset.labeled_idxs = self.labeled_idxs.copy()
        return new_dataset

In [10]:
# Instantiate the interface
unlabelled_pool = ActiveLearningDataset(train_dataset)
unlabelled_pool.initialize_labels(300) # 300 is the fixed amount for training the initial model, don't change it!

In [11]:
# If you wish to use the same initial data from unlabelled pool before trying a new sampling method, run this code!
# Get a copy of the unlabelled pool
pool = unlabelled_pool.copy_with_updates()

In [12]:
class OutputSorter:
    def __init__(self, predictions):
        self.predictions = predictions

    def sort_by_output(self):
        self.predictions.sort(key=lambda x: x[1])

    def get_smallest(self, num):
        return [idx for idx, _ in self.predictions[:num]]

### Modeling

In [13]:
# Interface of model
class Net:
    def __init__(self, net, discriminator, params, device):
        self.net = net
        self.discriminator = discriminator
        self.params = params
        self.device = device
        self.bce_loss = nn.BCELoss()
        
    def train(self, data, validation_dataset):
        n_epoch = self.params['n_epoch']
        self.clf = self.net(self.params['num_classes']).to(device)
        # self.clf = models.resnet18(pretrained=False)
        # num_features = self.clf.fc.in_features
        # self.clf.fc = nn.Linear(num_features, 6)

        self.clf.train()
        optimizer = optim.SGD(self.clf.parameters(), momentum=0.9, **self.params['optimizer_args'])
        
        loader = DataLoader(data, shuffle=True, **self.params['train_args'])
        val_loader = DataLoader(validation_dataset, shuffle=False, batch_size=128)
        
        epoch_losses = []
        val_accuracies = []  # 记录验证集准确率
        val_epochs = []  # 记录进行验证的epoch
        
        for epoch in tqdm(range(1, n_epoch+1), ncols=100):
            total_loss = 0
            for batch_idx, (_, x, y, _) in enumerate(loader):
                x, y = x.to(self.device), y.to(self.device)
                optimizer.zero_grad()
                out = self.clf(x)
                loss = F.cross_entropy(out, y)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            avg_loss = total_loss / len(loader)
            epoch_losses.append(avg_loss)
            print(f"epoch {epoch+1}, loss_train {avg_loss}")

            # 每五个epoch进行一次验证
            if (epoch+1) % 10 == 0:
                # 验证过程
                self.clf.eval()  # 切换到评估模式
                correct = 0
                total = 0
                with torch.no_grad():
                    for batch in val_loader:
                        idx, x, y, img_path = batch
                        x, y = x.to(self.device), y.to(self.device)
                        out = self.clf(x)
                        predictions = out.argmax(dim=1, keepdim=True)
                        correct += predictions.eq(y.view_as(predictions)).sum().item()
                        total += y.size(0)

                val_accuracy = correct / total
                val_accuracies.append(val_accuracy)
                val_epochs.append(epoch+1)  # 记录进行验证的epoch
                
                print(f"epoch {epoch+1}, acc_val {val_accuracy}")
        
        return epoch_losses, val_accuracies, val_epochs
            
    def train_dsc(self, data, epoch_dsc):
        # n_epoch = int(self.params['n_epoch'] / 2)
        n_epoch = epoch_dsc
        self.dsc = self.discriminator(self.params['num_classes']).to(device)
        self.dsc.train()
        optimizer = optim.Adam(self.dsc.parameters(), lr=5e-6)
        loader = DataLoader(data, shuffle=True, **self.params['train_args'])
        
        epoch_losses = []
        
        for epoch in tqdm(range(1, n_epoch+1), ncols=100):
            total_loss = 0
            for batch_idx, (x, y) in enumerate(loader):
                x, y = x.to(self.device), y.to(self.device).float()
                y = y.view(-1, 1)
                optimizer.zero_grad()
                out = self.dsc(x)
                loss = self.bce_loss(out, y)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            avg_loss = total_loss / len(loader)
            epoch_losses.append(avg_loss)
            
        return epoch_losses
    
    def predict_dsc(self, unlabeled_data, unlabeled_idx):
        # 确保模型处于评估模式
        self.clf.eval()
        self.dsc.eval()
        loader = DataLoader(unlabeled_data, batch_size=64, shuffle=False)
        
        prediction_results = []

        # 禁止计算梯度
        with torch.no_grad():
            for idx, x, _, _ in loader:
                x = x.to(self.device)  # 确保数据在正确的设备上

                # 使用第一个模型进行预测
                clf_outputs = self.clf(x)

                # 使用第二个模型对第一个模型的输出进行预测
                dsc_outputs = self.dsc(clf_outputs)
                
                for i in range(len(idx)):
                    prediction_results.append((idx[i], dsc_outputs[i]))
        return prediction_results
    
    def make_dataset(self, labeled_data, unlabeled_data):
        self.clf.eval()
        labeled_loader = DataLoader(labeled_data, shuffle=False, **self.params['test_args'])
        unlabeled_loader = DataLoader(unlabeled_data, shuffle=False, **self.params['test_args'])

        dataset_features = []
        dataset_labels = []

        with torch.no_grad():
            # 处理有标签的数据
            for x in labeled_loader:
                x = x[1].to(self.device) 
                outputs = self.clf(x) 
                # TODO
                outputs = F.softmax(outputs, dim=1)  # 应用softmax
                dataset_features.extend(outputs.cpu().numpy()) 
                dataset_labels.extend([1]*x.size(0)) 

            
            for x in unlabeled_loader:
                x = x[1].to(self.device)
                outputs = self.clf(x)
                # TODO
                outputs = F.softmax(outputs, dim=1)  # 应用softmax
                dataset_features.extend(outputs.cpu().numpy()) 
                dataset_labels.extend([0]*x.size(0))
        
        return DscDataset(dataset_features, dataset_labels)
        
        
    def validate(self, data):
        self.clf.eval()
        loader = DataLoader(data, shuffle=False, **self.params['test_args'])
        preds_list = []
        paths_list = []
        with torch.no_grad():
            for _, x, _, path in loader:
                x = x.to(self.device)
                out = self.clf(x)
                pred = out.max(1)[1]
                preds_list.append(pred.cpu())
                paths_list.extend(path)

        preds = torch.cat(preds_list)
        return preds, paths_list

    
    def predict(self, data):
        self.clf.eval()
        loader = DataLoader(data, shuffle=False, **self.params['test_args'])
        preds_list = []
        paths_list = []
        with torch.no_grad():
            for x, path in loader:
                x = x.to(self.device)
                out = self.clf(x)
                pred = out.max(1)[1]
                preds_list.append(pred.cpu())
                paths_list.extend(path)

        preds = torch.cat(preds_list)
        return preds, paths_list

    def predict_prob(self, data):
        self.clf.eval()
        loader = DataLoader(data, shuffle=False, **self.params['test_args'])
        probs_list = []
        with torch.no_grad():
            for x, _, paths in loader:
                x = x.to(self.device)
                outputs = self.clf(x)
                probs = torch.softmax(outputs, dim=1)
                probs_list.append(probs.cpu())

        probs = torch.cat(probs_list)
        return probs

In [14]:
import torch.nn.init as init
def kaiming_init(m):
    if isinstance(m, (nn.Linear, nn.Conv2d)):
        init.kaiming_normal(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0)
    elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
        m.weight.data.fill_(1)
        if m.bias is not None:
            m.bias.data.fill_(0)

In [15]:
class Discriminator(nn.Module):
    """Adversary architecture(Discriminator) for WAE-GAN."""
    def __init__(self, z_dim=10):
        super(Discriminator, self).__init__()
        self.z_dim = z_dim
        self.net = nn.Sequential(
            nn.Linear(z_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        self.weight_init()

    def weight_init(self):
        for block in self._modules:
            for m in self._modules[block]:
                kaiming_init(m)

    def forward(self, z):
        return self.net(z)

In [16]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [17]:
class ResNet(nn.Module):
    def __init__(self, num_classes=6):
        block = BasicBlock
        layers=[1, 1, 1, 1]  # 减少网络深度
        super(ResNet, self).__init__()
        self.in_channels = 32  # 减少参数数量
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)  # 使用更小的卷积核
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 32, layers[0])
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)  # 添加Dropout层
        self.fc = nn.Linear(256 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [18]:
# Interface of sampling method
class Strategy:
    def __init__(self, dataset, net):
        self.dataset = dataset
        self.net = net

    def query(self, n):
        pass

    def update(self, pos_idxs, neg_idxs=None):
        self.dataset.labeled_idxs[pos_idxs] = True
        if neg_idxs:
            self.dataset.labeled_idxs[neg_idxs] = False
            
    def update_by_path(self, path):
        for i in path:
            self.dataset.set_labeled_true(i)

    def train(self, epoch_dsc, validation_dataset):
        labeled_data, labeled_idxs = self.dataset.get_labeled_data()
        epoch_losses_clf, val_accuracies_clf, val_epochs_clf = self.net.train(labeled_data, validation_dataset)
        labeled_len = len(labeled_data)
        unlabel_len = 2*labeled_len
        unlabeled_data, unlabeled_idxs = self.dataset.get_required_unlabeled_data(unlabel_len)
        dsc_dataset = self.net.make_dataset(labeled_data, unlabeled_data)
        epoch_losses_des = self.net.train_dsc(dsc_dataset, epoch_dsc)

        return epoch_losses_clf, val_accuracies_clf, val_epochs_clf, epoch_losses_des
    
    def validate(self, data):
        preds = self.net.validate(data)
        return preds

    def predict(self, data):
        preds = self.net.predict(data)
        return preds

    def predict_prob(self, data):
        probs = self.net.predict_prob(data)
        return probs

In [19]:
# Try different sampling method or design your own sampling method here
class DBAL(Strategy):
    def __init__(self, dataset, net):
        super(DBAL, self).__init__(dataset, net)

    def query(self, n):
        unlabeled_data, unlabeled_idx = self.dataset.get_unlabeled_data()
        prediction = self.net.predict_dsc(unlabeled_data, unlabeled_idx)
        sorter = OutputSorter(prediction)
        sorter.sort_by_output() 
        smallest_indices = sorter.get_smallest(300) 
        return smallest_indices

In [20]:
def calculate_accuracy(strategy, valid_dataset):
    strategy.net.clf.eval()
    correct = 0
    total = 0
    labels = [label for _, _, label, _ in valid_dataset]
    labels = torch.tensor(labels)
    predicted, _ = strategy.validate(valid_dataset)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

### Plotting

In [21]:
def plot_training_curves(epoch_losses_clf, val_accuracies_clf, val_epochs_clf, epoch_losses_des, rd):
    # 绘制分类器的损失曲线
    n_epochs_clf = len(epoch_losses_clf)
    plt.figure(figsize=(12, 5))
    plt.plot(range(1, n_epochs_clf + 1), epoch_losses_clf, color='tab:red', marker='o', linestyle='-', label='Classifier Loss')
    plt.title('Classifier Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.savefig(f'{rd}_classifier_training_loss.png')  # 保存分类器的损失图
    plt.close()  # 关闭图形

    # 绘制分类器的验证准确率曲线
    n_epochs_val = len(val_accuracies_clf)
    plt.figure(figsize=(12, 5))
    plt.plot(val_epochs_clf, val_accuracies_clf, color='tab:blue', marker='x', linestyle='-', label='Validation Accuracy')
    plt.title('Classifier Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.savefig(f'{rd}_classifier_validation_accuracy.png')  # 保存分类器的验证准确率图
    plt.close()  # 关闭图形

    # 绘制鉴别器的损失曲线
    n_epochs_des = len(epoch_losses_des)
    plt.figure(figsize=(12, 5))
    plt.plot(range(1, n_epochs_des + 1), epoch_losses_des, color='tab:green', marker='s', linestyle='-', label='Discriminator Loss')
    plt.title('Discriminator Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.savefig(f'{rd}_discriminator_training_loss.png')  # 保存鉴别器的损失图
    plt.close()  # 关闭图形

### Training

In [22]:
### pool = unlabelled_pool.copy_with_updates()
# Try different hyperparameters
model_params = {
    'n_epoch': 400,
    'epoch_dsc': 50,
    'num_classes': 6,
    'optimizer_args': {'lr': 0.0005, 'weight_decay': 5e-6},
    'train_args': {'batch_size': 32},
    'test_args': {'batch_size': 256},
}
best_accuracy = 0.0  # initialize the best accuracy
best_model_random = None  # initialize the best model
# Random sampling
net = Net(ResNet, Discriminator, model_params, device)
strategy = DBAL(pool, net)

# round 0 accuracy
print("Round 0")
labeled_dataset, _ = pool.get_labeled_data()
print(f"Size of Dataset: {len(labeled_dataset)}")
# TODO
epoch_losses_clf, val_accuracies_clf, val_epochs_clf, epoch_losses_des = strategy.train(model_params['epoch_dsc'], validation_dataset)
# plot_training_curves(epoch_losses_clf, val_accuracies_clf, val_epochs_clf, epoch_losses_des, rd=0)

accuracy = calculate_accuracy(strategy, validation_dataset)
print(f"Round 0 validation accuracy: {accuracy}")

for rd in range(1, 4): # 4 = rounds + 1. I perform the 3 rounds of sampling. You can try different rounds!
    print(f"Round {rd}")

    # query
    query_idxs = strategy.query(n=300)
    print(query_idxs)
    # update labels
    strategy.update(query_idxs)
    labeled_dataset, _ = pool.get_labeled_data()
    print(f"Size of Dataset: {len(labeled_dataset)}")
    # TODO
    epoch_losses_clf, val_accuracies_clf, val_epochs_clf, epoch_losses_des = strategy.train(model_params['epoch_dsc'], validation_dataset)
    # plot_training_curves(epoch_losses_clf, val_accuracies_clf, val_epochs_clf, epoch_losses_des, rd=rd)

    # calculate accuracy
    accuracy = calculate_accuracy(strategy, validation_dataset)
    print(f"Round {rd} validation accuracy: {accuracy}")

    # check if the current model is the best
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        # deep copy currently best model
        best_model_random = copy.deepcopy(strategy.net.clf)
        print(f"New best model saved with accuracy: {accuracy}")

Round 0
Size of Dataset: 300


  0%|▏                                                              | 1/400 [00:02<14:47,  2.22s/it]

epoch 2, loss_train 1.9035237431526184


  0%|▎                                                              | 2/400 [00:04<13:10,  1.99s/it]

epoch 3, loss_train 1.8283197641372682


...

epoch 62, loss_train 0.8428418981401544


 16%|█████████▌                                                    | 62/400 [08:10<44:43,  7.94s/it]

epoch 63, loss_train 0.7533343693143443


 16%|█████████▌                                                    | 62/400 [08:11<44:41,  7.93s/it]


KeyboardInterrupt: 

In [24]:
predictions = []
image_names = []

# Put the best model back into strategy to predict
strategy.net.clf = best_model_random
predicted, paths = strategy.predict(test_dataset)
predictions.extend(predicted.cpu().numpy())
image_names.extend(paths)


submission_df = pd.DataFrame({
    'ID': image_names,
    'label': predictions
})


submission_df['ID'] = submission_df['ID'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
submission_df = submission_df.sort_values(by='ID').reset_index(drop=True)

# Save submission file
submission_df.to_csv('submission.csv', index=False)